## Problem 1

In [10]:
def read_input(filename):
    f = open(f'../inputs/{filename}.txt', 'r')

    values = {}
    gates = {}
    dependencies = {}
    while True:
        line = f.readline()
        if line == '\n':
            break
        values[line[:3]] = int(line[5])
    while True:
        line = f.readline()
        if line == '':
            break
        arr = line.split(' ')
        x, op, y, z = arr[0], arr[1], arr[2], arr[4].strip()
        gates[z] = (x, y, op)
        if x not in dependencies:
            dependencies[x] = []
        if y not in dependencies:
            dependencies[y] = []
        dependencies[x].append(z)
        dependencies[y].append(z)

    f.close()
    return (values, gates, dependencies)

In [38]:
def compute_gate(val_x, val_y, op):
    if op == 'OR':
        return val_x | val_y
    if op == 'AND':
        return val_x & val_y
    if op == 'XOR':
        return val_x ^ val_y

def compute_z(var_name, value):
    index = int(var_name[1:])
    return value * pow(2, index)

def solve1(input_filename):
    (values, gates, dependencies) = read_input(input_filename)
    
    z = 0
    L = list(values.keys())
    while len(L) > 0:
        current = L.pop(0)
        if current.startswith('z'):
            z += compute_z(current, values[current])
        
        if current in dependencies:
            for dep in dependencies[current]:
                (x, y, op) = gates[dep]
                if dep not in values and x in values and y in values:
                    L.append(dep)
                    values[dep] = compute_gate(values[x], values[y], op)
    
    return z

## Problem 2

In [107]:
def two_char(ind):
    str_ind = str(ind)
    if len(str_ind) == 1:
        return '0' + str_ind
    return str_ind

def print_wrong_bits(input_filename):
    (values, gates, dependencies) = read_input(input_filename)
    # values = all x, y are zeros

    for i in range(45):
        mod_values = values.copy()
        mod_values['x'+two_char(i)] = 1
        
        (x, y, z) = compute_result(mod_values, gates, dependencies)
        if z != x + y:
            print(i)
            print(bin(x))
            print(bin(y))
            print(bin(z))

def compute_result(values, gates, dependencies):
    z = 0
    x_val = 0
    y_val = 0
    L = list(values.keys())
    while len(L) > 0:
        current = L.pop(0)
        if current.startswith('z'):
            z += compute_z(current, values[current])
        if current.startswith('x'):
            x_val += compute_z(current, values[current])
        if current.startswith('y'):
            y_val += compute_z(current, values[current])
        if current in dependencies:
            for dep in dependencies[current]:
                (x, y, op) = gates[dep]
                if dep not in values and x in values and y in values:
                    L.append(dep)
                    values[dep] = compute_gate(values[x], values[y], op)

    return (x_val, y_val, z)

In [114]:
op_str = {
    'AND': ' & ',
    'OR': ' | ',
    'XOR': ' ^ '
}

def calc_str(wire, gates):
    if wire.startswith('x') or wire.startswith('y'):
        return wire
    (x, y, op) = gates[wire]
    str_x, str_y = calc_str(x, gates), calc_str(y, gates)
    if len(str_x) < len(str_y) or (len(str_x) == len(str_y) and str_x > str_y):
        return '(' + str_x + op_str[op] + str_y + ')'
    return '(' + str_y + op_str[op] + str_x + ')'
    

def print_dep(input_filename, wire):
    (values, gates, dependencies) = read_input(input_filename)

    print(wire + ' = ' + calc_str(wire, gates))